In [122]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
import pandas as pd

In [123]:
df = pd.read_csv("fully_features.csv")

In [124]:
df.shape

(402, 13)

In [125]:
labels = pd.read_csv("all_our_annotatations.csv", sep=":::")

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [126]:
labels.head()

,sequence,char1,char2,manualLabel,sid,Sentence
0,1,0,3,p,s0,Many_-1 years_-1 after_-1 king_-1 Agamemnon_3 ...
1,1,0,3,p,s9,"Together_-1 ,_-1 Orestes_0 and_-1 Electra_1 pl..."
2,1,0,3,p,s10,With_-1 the_-1 eager_-1 support_-1 of_-1 the_-...
3,1,0,3,p,s12,"Unexpectedly_-1 ,_-1 Clytamnestra_5 comes_-1 t..."
4,1,0,3,p,s13,He_3 claims_-1 to_-1 be_-1 a_-1 stranger_-1 be...


In [127]:
labels = labels[:402]
labels.shape

(402, 6)

In [128]:
all_data = df.merge(labels)

In [129]:
all_data.head()

,Unnamed: 0,sequence,sid,pos_acts_toeachother,neg_acts_toeachother,pos_acts_together,neg_acts_together,pos_char1_acts,neg_char1_acts,pos_char2_acts,neg_char2_acts,pos,neg,char1,char2,manualLabel,Sentence
0,0,1,s0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,3,p,Many_-1 years_-1 after_-1 king_-1 Agamemnon_3 ...
1,1,1,s9,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3,p,"Together_-1 ,_-1 Orestes_0 and_-1 Electra_1 pl..."
2,2,1,s10,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,3,p,With_-1 the_-1 eager_-1 support_-1 of_-1 the_-...
3,3,1,s12,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3,p,"Unexpectedly_-1 ,_-1 Clytamnestra_5 comes_-1 t..."
4,4,1,s13,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,3,p,He_3 claims_-1 to_-1 be_-1 a_-1 stranger_-1 be...


In [130]:
all_data = all_data.drop(['Unnamed: 0', 'char1', 'char2', 'Sentence'], axis=1)

In [131]:
X = all_data.drop(['manualLabel', 'sequence', 'sid'], axis=1)
Y = all_data.manualLabel

Y.loc[Y == 'p'] = 1
Y.loc[Y == 'n'] = 2
Y.loc[Y == 'u'] = 3

Y=Y.astype('int')

X_train = X[:300]
y_train = Y[:300]

X_test = X[300:]
y_test = Y[300:]

/usr/local/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [132]:
logreg = linear_model.LogisticRegression(C=1e5)

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X_train, y_train)

pred = logreg.predict(X_test)

In [139]:
results = pd.DataFrame()
print(pred.shape)
print(y_test.shape)
results['pred'] = pred
results['y_test'] = np.array(y_test)
results['correct'] = 0

(102,)
(102,)


In [145]:
results.head()

,pred,y_test,correct
0,1,2,0
1,1,1,0
2,1,1,0
3,1,1,0
4,2,1,0


In [159]:
results['correct'] = (results['pred'] == results['y_test']).astype(int)

In [160]:
results.head()

,pred,y_test,correct
0,1,2,0
1,1,1,1
2,1,1,1
3,1,1,1
4,2,1,0


In [164]:
results['correct'].sum()/results['correct'].count()

0.8333333333333334